In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ultranest
import read_beam_FEKO as rf
import scipy.interpolate
import scipy.optimize
%matplotlib notebook

In [2]:
o = rf.read_beam_FEKO('nominal.out',0)

 Frequency in Hz:               FREQ =    2.50000E+07

 Frequency in Hz:               FREQ =    2.60000E+07

 Frequency in Hz:               FREQ =    2.70000E+07

 Frequency in Hz:               FREQ =    2.80000E+07

 Frequency in Hz:               FREQ =    2.90000E+07

 Frequency in Hz:               FREQ =    3.00000E+07

 Frequency in Hz:               FREQ =    3.10000E+07

 Frequency in Hz:               FREQ =    3.20000E+07

 Frequency in Hz:               FREQ =    3.30000E+07

 Frequency in Hz:               FREQ =    3.40000E+07

 Frequency in Hz:               FREQ =    3.50000E+07

 Frequency in Hz:               FREQ =    3.60000E+07

 Frequency in Hz:               FREQ =    3.70000E+07

 Frequency in Hz:               FREQ =    3.80000E+07

 Frequency in Hz:               FREQ =    3.90000E+07

 Frequency in Hz:               FREQ =    4.00000E+07

 Frequency in Hz:               FREQ =    4.10000E+07

 Frequency in Hz:               FREQ =    4.20000E+07

 Frequency

In [8]:
# Define theta as an array from [90 to 0]. phi is from [0 to 359], freq from [25 to 125 MHz]
theta = 90 - o[2]
phi = o[1]
freq = o[0]
gain = o[5]
xx, yy = np.meshgrid(theta, phi)
theta = xx
phi = yy
#xx is for theta, and yy is for phi

In [4]:
#The fundamental TDH functions
#Acos,Asin taken theta in degree
def Acos(R, theta, miu, nu):
    part1 = np.cos(miu*theta*np.pi/180)*(R/(sigma/1.43))**miu
    part2 = P(miu,nu,R**2/(sigma/1.43)**2)
    part3 = np.exp(-R**2/(4*(sigma/1.43)**2))
    return part1*part2*part3

def Asin(R, theta, miu, nu):
    part1 = np.sin(miu*theta*np.pi/180)*(R/(sigma/1.43))**miu
    part2 = P(miu,nu,R**2/(sigma/1.43)**2)
    part3 = np.exp(-R**2/(4*(sigma/1.43)**2))
    return part1*part2*part3

def P(miu,nu,u):
    terms = []
    p = 0
    while p <= nu:
        up = np.math.factorial(miu + nu)*np.math.factorial(nu)
        down = np.math.factorial(miu + p)*np.math.factorial(p)*np.math.factorial(nu-p)
        term = (-2)**(nu-p)* up/down * u**p
        terms.append(term)
        p += 1
    terms = np.array(terms)
    return np.sum(terms,axis = 0)

In [9]:
#Normalization
def normalize(array):
    q = np.max(array)
    return array/q

def TDH(rank):
    """return matrix A = [f1, f2, f3, ...] for rank 0 to rank TDH functions"""
    zz00c = Acos(theta, phi, 0, 0)
    zz00c2d = zz00c.flatten()
    array = normalize(zz00c2d)
    r = 1
    while r <= rank:
        m = 0
        while m <= r:
            if (r - m)%2 == 0:
                n = (r - m)/2
                if m == 0:
                    cosine = Acos(theta,phi,m,n)
                    cosine2d = cosine.flatten()
                    ncos = normalize(cosine2d)
                    array = np.vstack((array, ncos))
                    m += 2
                else:
                    cosine = Acos(theta,phi,m,n)
                    cosine2d = cosine.flatten()
                    ncos = normalize(cosine2d)
                    sine = Asin(theta,phi,m,n)
                    sine2d = sine.flatten()
                    nsin = normalize(sine2d)
                    array = np.vstack((array, ncos))
                    array = np.vstack((array, nsin))
                    m += 2
            else:
                m += 1
        r += 1
    matrix = np.matrix(array).T
    return matrix

def model(rank, coe):
    A = TDH(rank)
    a = np.matrix(coe).T
    return np.matmul(A,a)

def simulation(coe, sigma):
    rank = 0
    count = 1
    length = len(coe) - 1
    while length > count:
        rank += 1
        count += rank + 1
    mo = model(rank,coe)
    noise = np.random.default_rng().normal(0.0, sigma*np.max(mo),360*91)
    noi = np.matrix(noise).T
    data = mo + noi
    return data

def solve(A, data):
    AT = A.T
    ATA = np.matmul(AT,A)
    ATAinverse = np.linalg.inv(ATA)
    half = np.matmul(ATAinverse,AT)
    x = np.matmul(half,data)
    return x

def residual(A,data):
    x = solve(A, data)
    AT = A.T
    fittingmodel = np.matmul(A, x)
    r = data - fittingmodel
    return r

def residual2D(A,data):
    r = residual(A,data)
    r2D = r.reshape(360,91)
    return r2D

def covariance(A,data):
    AT = A.T
    ATA = np.matmul(AT,A)
    ATAinverse = np.linalg.inv(ATA)
    r = residual(A,data)
    rT = r.T
    rTr = np.matmul(rT,r)
    s = rTr/((len(data))-len(AT)) 
    '''Originally: s = rTr/((len(data))-10). I think the 10 is weird.'''
    s2 = s.item(0)
    covariance = s2*ATAinverse
    return covariance

25MHz

In [12]:
# 2D Gaussian data fitting using UltraNest
def Gaussian2D(A, sigx, sigy):
    return A * np.exp(-(theta*np.pi/180*np.cos(phi*np.pi/180))**2/2/sigx**2 - (theta*np.pi/180*np.sin(phi*np.pi/180))**2/2/sigy**2) 

# Define model to simulate data
def model1(paras):
    A, sigx, sigy = paras
    return Gaussian2D(A,sigx,sigy).T

sigma = np.std(gain[0])
param_names = ['A', 'sigx', 'sigy']
def my_prior_transform(cube):
    params = cube.copy()

    # transform location parameter: uniform prior
    lo = 0
    hi = 10
    params[0] = cube[0] * (hi - lo) + lo

    lo = 0
    hi = 2
    params[1] = cube[1] * (hi - lo) + lo

    lo = 0
    hi = 2
    params[1] = cube[1] * (hi - lo) + lo
    
    return params
def my_likelihood(params):
    mo = model1(params)
    diff = mo - gain[0]
    return -0.5 * np.sum((diff/sigma)**2)

In [13]:
sampler = ultranest.ReactiveNestedSampler(param_names, my_likelihood, my_prior_transform)
result = sampler.run(min_num_live_points=200)
sampler.print_results()

[ultranest] Sampling 200 live points from prior ...


[ultranest] Explored until L=-2e+02  .79 [-185.8414..-185.8412]*| it/evals=4340/8820 eff=50.3480% N=200 0  0    0 0  0  
[ultranest] Likelihood function evaluations: 8834
[ultranest]   logZ = -202.9 +- 0.2342
[ultranest] Effective samples strategy satisfied (ESS = 969.1, need >400)
[ultranest] Posterior uncertainty strategy is satisfied (KL: 0.45+-0.09 nat, need <0.50 nat)
[ultranest] Evidency uncertainty strategy wants 198 minimum live points (dlogz from 0.18 to 0.61, need <0.5)
[ultranest]   logZ error budget: single: 0.28 bs:0.23 tail:0.01 total:0.23 required:<0.50
[ultranest] done iterating.

logZ = -202.962 +- 0.613
  single instance: logZ = -202.962 +- 0.279
  bootstrapped   : logZ = -202.882 +- 0.613
  tail           : logZ = +- 0.010
insert order U test : converged: True correlation: inf iterations

    A                   : 0.5285│ ▁▁▁▁▁▁▁▁▂▃▄▄▄▅▆▇▇▇▆▆▆▆▅▃▄▃▂▁▁▁▁▁▁▁ ▁▁ │0.5430    0.5353 +- 0.0019
    sigx                : 0.6437│ ▁▁▁▁▁▁▁▁▁▂▂▃▃▄▆▅▆▇▇▆▇▆▅▃▃▃▂▁▁▁▁▁▁▁▁▁▁ │0.6790  

In [14]:
sigma = np.sqrt(0.5353*0.6612)*180/np.pi
A = TDH(7)

In [15]:
data = np.matrix(gain[0].T.flatten())
coe = solve(A,data.T)

In [18]:
for i in enumerate(coe):
    print(i)

(0, matrix([[0.61847371]]))
(1, matrix([[-2.75855895e-05]]))
(2, matrix([[1.82601783e-06]]))
(3, matrix([[0.03015419]]))
(4, matrix([[-0.01061785]]))
(5, matrix([[-1.67127223e-06]]))
(6, matrix([[-3.88467275e-06]]))
(7, matrix([[-2.17903305e-07]]))
(8, matrix([[9.8159873e-08]]))
(9, matrix([[-5.17776008e-08]]))
(10, matrix([[-0.06975951]]))
(11, matrix([[0.01705658]]))
(12, matrix([[1.09807664e-06]]))
(13, matrix([[-9.62955447e-05]]))
(14, matrix([[-1.58604081e-08]]))
(15, matrix([[1.25121872e-05]]))
(16, matrix([[-4.10461458e-07]]))
(17, matrix([[-4.39251374e-08]]))
(18, matrix([[6.46552514e-08]]))
(19, matrix([[-1.04359117e-10]]))
(20, matrix([[7.06402246e-09]]))
(21, matrix([[-0.01815894]]))
(22, matrix([[-0.00093073]]))
(23, matrix([[3.39663617e-07]]))
(24, matrix([[-0.0002065]]))
(25, matrix([[-1.74689987e-08]]))
(26, matrix([[5.16938222e-07]]))
(27, matrix([[1.89670646e-08]]))
(28, matrix([[7.82785648e-06]]))
(29, matrix([[-6.73606587e-08]]))
(30, matrix([[-6.50339126e-08]]))
(31

In [20]:
np.sort(abs(coe),axis = None)

matrix([[1.04359117e-10, 1.79103567e-09, 2.40160981e-09, 4.97459490e-09,
         7.06402246e-09, 1.58604081e-08, 1.74689987e-08, 1.89670646e-08,
         2.39950321e-08, 4.29448608e-08, 4.39251374e-08, 5.17776008e-08,
         6.46552514e-08, 6.50339126e-08, 6.73606587e-08, 9.81598730e-08,
         2.17903305e-07, 3.39663617e-07, 4.10461458e-07, 5.16938222e-07,
         1.09807664e-06, 1.67127223e-06, 1.82601783e-06, 3.88467275e-06,
         7.82785648e-06, 1.25121872e-05, 2.75855895e-05, 9.62955447e-05,
         2.06499554e-04, 9.30725969e-04, 1.06178546e-02, 1.70565765e-02,
         1.81589373e-02, 3.01541874e-02, 6.97595103e-02, 6.18473714e-01]])

In [40]:
maxind25 = []
for i in range(7):
    maxind25.append(np.where(abs(coe) == np.sort(abs(coe),axis = None).T[35-i])[0][0])

In [42]:
order = ['cos00','cos10','sin10','cos01','cos20','sin20','cos11','sin11','cos30','sin30',
        'cos02','cos21','sin21','cos40','sin40','cos12','sin12','cos31','sin31','cos50',
        'sin50','cos03','cos22','sin22','cos41','sin41','cos60','sin60','cos13','sin13',
        'cos32','sin32','cos51','sin51','cos70','sin70'
        ]

In [43]:
for i in range(7):
    print(order[maxind25[i]])

cos00
cos02
cos01
cos03
cos21
cos20
cos22


125MHz

In [44]:
# 2D Gaussian data fitting using UltraNest
def Gaussian2D(A, sigx, sigy):
    return A * np.exp(-(theta*np.pi/180*np.cos(phi*np.pi/180))**2/2/sigx**2 - (theta*np.pi/180*np.sin(phi*np.pi/180))**2/2/sigy**2) 

# Define model to simulate data
def model1(paras):
    A, sigx, sigy = paras
    return Gaussian2D(A,sigx,sigy).T

sigma = np.std(gain[100])
param_names = ['A', 'sigx', 'sigy']
def my_prior_transform(cube):
    params = cube.copy()

    # transform location parameter: uniform prior
    lo = 0
    hi = 10
    params[0] = cube[0] * (hi - lo) + lo

    lo = 0
    hi = 2
    params[1] = cube[1] * (hi - lo) + lo

    lo = 0
    hi = 2
    params[1] = cube[1] * (hi - lo) + lo
    
    return params
def my_likelihood(params):
    mo = model1(params)
    diff = mo - gain[100]
    return -0.5 * np.sum((diff/sigma)**2)

In [45]:
sampler = ultranest.ReactiveNestedSampler(param_names, my_likelihood, my_prior_transform)
result = sampler.run(min_num_live_points=200)
sampler.print_results()

[ultranest] Sampling 200 live points from prior ...


[ultranest] Explored until L=-2e+02  .29 [-177.3382..-177.3377]*| it/evals=4000/8108 eff=50.5817% N=200    0  
[ultranest] Likelihood function evaluations: 8111
[ultranest]   logZ = -192.7 +- 0.1858
[ultranest] Effective samples strategy satisfied (ESS = 950.1, need >400)
[ultranest] Posterior uncertainty strategy is satisfied (KL: 0.46+-0.09 nat, need <0.50 nat)
[ultranest] Evidency uncertainty strategy is satisfied (dlogz=0.19, need <0.5)
[ultranest]   logZ error budget: single: 0.26 bs:0.19 tail:0.01 total:0.19 required:<0.50
[ultranest] done iterating.

logZ = -192.717 +- 0.426
  single instance: logZ = -192.717 +- 0.264
  bootstrapped   : logZ = -192.736 +- 0.426
  tail           : logZ = +- 0.010
insert order U test : converged: True correlation: inf iterations

    A                   : 5.037 │ ▁  ▁▁▁▁▁▁▁▁▂▃▄▅▆▇▆▇▇▆▅▅▅▄▃▂▂▁▁▁▁▁▁▁ ▁ │5.248     5.145 +- 0.024
    sigx                : 0.3061│ ▁▁▁▁▁▁▁▁▂▂▃▃▃▅▆▅▅▇▇▇▆▇▅▅▄▄▂▁▂▁▁▁▁▁▁▁▁ │0.3237    0.3149 +- 0.0024
    sigy               

In [46]:
sigma = np.sqrt(0.3149*0.8241)*180/np.pi
A = TDH(7)

In [47]:
data = np.matrix(gain[0].T.flatten())
coe = solve(A,data.T)

In [50]:
maxind125 = []
for i in range(10):
    maxind125.append(np.where(abs(coe) == np.sort(abs(coe),axis = None).T[35-i])[0][0])

In [51]:
for i in range(10):
    print(order[maxind125[i]])

cos00
cos01
cos20
cos21
cos03
cos22
cos02
cos41
cos10
cos11


In [54]:
maxind125

[0, 3, 4, 11, 21, 22, 10, 24, 1, 6]

In [52]:
for i in enumerate(coe):
    print(i)

(0, matrix([[0.69992642]]))
(1, matrix([[-3.00483321e-05]]))
(2, matrix([[2.11929754e-06]]))
(3, matrix([[0.09242058]]))
(4, matrix([[-0.01757451]]))
(5, matrix([[-2.22693128e-06]]))
(6, matrix([[-1.601057e-05]]))
(7, matrix([[3.86357867e-07]]))
(8, matrix([[1.23667456e-07]]))
(9, matrix([[-8.04114295e-08]]))
(10, matrix([[0.0022873]]))
(11, matrix([[0.01492401]]))
(12, matrix([[4.28541196e-07]]))
(13, matrix([[-7.77028004e-06]]))
(14, matrix([[-9.59885445e-09]]))
(15, matrix([[1.195796e-06]]))
(16, matrix([[-3.0808603e-07]]))
(17, matrix([[3.80910437e-08]]))
(18, matrix([[1.54149991e-08]]))
(19, matrix([[3.62232213e-10]]))
(20, matrix([[7.55243813e-09]]))
(21, matrix([[-0.01329041]]))
(22, matrix([[0.006672]]))
(23, matrix([[5.17244535e-07]]))
(24, matrix([[-0.00018223]]))
(25, matrix([[-2.08074459e-08]]))
(26, matrix([[3.41475558e-07]]))
(27, matrix([[2.17383182e-08]]))
(28, matrix([[4.85708995e-06]]))
(29, matrix([[-1.83124781e-07]]))
(30, matrix([[-2.88688445e-08]]))
(31, matrix([[

In [53]:
np.sort(abs(coe),axis = None)

matrix([[3.62232213e-10, 3.14177431e-09, 4.71295918e-09, 7.55243813e-09,
         9.28750022e-09, 9.59885445e-09, 1.54149991e-08, 2.08074459e-08,
         2.17383182e-08, 2.54260540e-08, 2.88688445e-08, 3.80910437e-08,
         4.15191159e-08, 8.04114295e-08, 1.23667456e-07, 1.83124781e-07,
         3.08086030e-07, 3.41475558e-07, 3.86357867e-07, 4.28541196e-07,
         5.17244535e-07, 1.19579600e-06, 2.11929754e-06, 2.22693128e-06,
         4.85708995e-06, 7.77028004e-06, 1.60105700e-05, 3.00483321e-05,
         1.82234951e-04, 2.28730439e-03, 6.67199816e-03, 1.32904119e-02,
         1.49240075e-02, 1.75745076e-02, 9.24205819e-02, 6.99926423e-01]])

Doesn't match up for 125MHz at all. For 25 MHz, match up for most of them.